### How accurate is analyst recommendation for stocks?

And if they are somewhat accurate, what is a typical time horizon for the momentum to halt?

In [138]:
%matplotlib inline
import yfinance as yf
import pandas as pd

In [139]:
# unique actions for each firm
ticker = yf.Ticker("BA")
ticker.recommendations.groupby("Firm")["Action"].apply(set)

Firm
Argus                                  {main, down}
Argus Research                                 {up}
BB&T Capital                             {down, up}
Baird                            {main, down, init}
Bank of America              {main, down, up, reit}
Barclays                               {main, down}
Berenberg                                    {down}
Bernstein                                    {main}
Buckingham                         {main, down, up}
Buckingham Research                {main, down, up}
CFRA                                         {down}
CRT Capital                                  {main}
Canaccord Genuity                {main, down, init}
Citi                                         {main}
Citigroup                                    {main}
Cowen & Co.                            {main, down}
Credit Suisse          {main, reit, up, init, down}
DA Davidson                                  {main}
Deutsche Bank                                {main}
Drexel 

In [140]:
# Unique To Grade for each firm
ticker.recommendations.groupby("Firm")["To Grade"].apply(set)

Firm
Argus                                   {Buy, Hold}
Argus Research                                {Buy}
BB&T Capital               {Buy, Hold, Underweight}
Baird                         {Neutral, Outperform}
Bank of America             {Neutral, Underperform}
Barclays                 {Overweight, Equal-Weight}
Berenberg                                    {Hold}
Bernstein                              {Outperform}
Buckingham                  {Neutral, Underperform}
Buckingham Research         {Neutral, Underperform}
CFRA                                          {Buy}
CRT Capital                                   {Buy}
Canaccord Genuity                       {Buy, Hold}
Citi                                          {Buy}
Citigroup                                     {Buy}
Cowen & Co.            {Outperform, Market Perform}
Credit Suisse                 {Neutral, Outperform}
DA Davidson                               {Neutral}
Deutsche Bank                                 {Buy}
Drexel 

In [141]:
ticker.calendar

,0,1
Earnings Date,2020-07-22 00:00:00,2020-07-27 00:00:00
Earnings Average,-2.22,-2.22
Earnings Low,-4.31,-4.31
Earnings High,-1.28,-1.28
Revenue Average,14409900000,14409900000
Revenue Low,12079000000,12079000000
Revenue High,16893000000,16893000000


In [142]:
# get latest analyst ratings
import dateutil

date_cutoff = dateutil.parser.parse("2020-03-01")

latest_rec = ticker.recommendations.sort_values('Date').groupby("Firm").tail(1)
latest_rec = lastest_rec[lastest_rec.index > date_cutoff]
latest_rec.head()

,Firm,To Grade,From Grade,Action
Date,,,,
2020-03-17 15:31:00,Deutsche Bank,Buy,,main
2020-03-31 10:30:02,Piper Sandler,Overweight,,main
2020-04-03 12:17:23,Raymond James,Strong Buy,,main
2020-04-28 12:41:37,Jefferies,Buy,,main
2020-04-30 11:01:29,Citigroup,Neutral,,main


In [143]:
def score_ratings(recs):
    mapping = {
        'Buy': 1,
        'Overweight': 1,
        'Strong Buy': 1,
        'Outperform': 1,
        'Neutral': 0,
        'Equal-Weight': 0,
        'Sell': -1,
        'Underweight': -1
    }
    score_series = recs['To Grade'].map(mapping)
    if score_series.hasnans:
        recs['score'] = recs['To Grade'].map(mapping)
        print(recs[pd.isnull(recs['score'])])
    return score_series.dropna().mean()

In [144]:
score_ratings(latest_rec)

0.9166666666666666

In [145]:
rdf = ticker.recommendations
# verifying they indeed don't have later actions
rdf[rdf['Firm'] == 'Argus Research']

,Firm,To Grade,From Grade,Action
Date,,,,
2012-05-21 07:51:00,Argus Research,Buy,,up
2020-03-27 13:03:50,Argus Research,Buy,Hold,up


In [146]:
ticker.financials

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [147]:
ticker.quarterly_financials

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [148]:
ticker.major_holders

,0,1
0,0.13%,% of Shares Held by All Insider
1,66.60%,% of Shares Held by Institutions
2,66.69%,% of Float Held by Institutions
3,2359,Number of Institutions Holding Shares


In [149]:
ticker.institutional_holders

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",41756605,2020-03-30,0.0740,6227580069
1,Blackrock Inc.,33252360,2020-03-30,0.0589,4959256970
2,Newport Trust Co,32725504,2020-03-30,0.0580,4880681666
3,Capital World Investors,30555090,2020-03-30,0.0541,4556986122
4,State Street Corporation,26146396,2020-03-30,0.0463,3899473499
5,Price (T.Rowe) Associates Inc,19483408,2020-03-30,0.0345,2905755469
6,Capital International Investors,10734838,2020-03-30,0.0190,1600993739
7,"Geode Capital Management, LLC",7206451,2020-03-30,0.0128,1074770102
8,Northern Trust Corporation,5539070,2020-03-30,0.0098,826096899
9,"Susquehanna International Group, LLP",5190304,2020-03-30,0.0092,774081938


In [150]:
ticker.balance_sheet

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [151]:
ticker.cashflow

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [152]:
ticker.earnings

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [153]:
ticker.quarterly_earnings

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [154]:
ticker.actions

,Dividends,Stock Splits
Date,,
1962-02-05,0.00823,0.0
1962-05-08,0.00823,0.0
1962-08-13,0.00823,0.0
1962-11-05,0.00823,0.0
1963-02-06,0.00823,0.0
...,...,...
2019-02-07,2.05500,0.0
2019-05-09,2.05500,0.0
2019-08-08,2.05500,0.0


### What are some highest rated stocks?

In [155]:
sp500df = pd.read_csv("sp500.csv")
sp500df.head()

,Symbol,Security,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [162]:
def get_analyst_score(sym, cutoff_date):
    try:
        ticker = yf.Ticker(sym)
        date_cutoff = dateutil.parser.parse(cutoff_date)
        
        if ticker.recommendations  ticker.recommendations.empty:
            return 0.0
        latest_rec = ticker.recommendations.sort_values('Date').groupby("Firm").tail(1)
        latest_rec = lastest_rec[lastest_rec.index > date_cutoff]
        if latest_rec.empty:
            return 0.0
    except IndexError:
        # yfinance fails to get data for symbol
        return 0.0
    except ImportError:
        # pd.read_html sometimes requires html5lib
        return 0.0
    except KeyError:
        # yfinance has no regularMarketOpen
        return 0.0
    return score_ratings(latest_rec)

In [163]:
# get_analyst_score('MMM', "2020-03-01")

sp500df['analyst_score'] = sp500df['Symbol'].apply(lambda x : get_analyst_score(x, "2020-03-01"))

AttributeError: 'NoneType' object has no attribute 'empty'

In [ ]:
sp500df.head()